In [1]:
#!pip install --ignore-installed --upgrade pandas

In [1]:
import pandas as pd
import numpy as np
import gensim
from sklearn.externals import joblib
from gensim import corpora, utils
from gensim.models.wrappers.dtmmodel import DtmModel
from collections import defaultdict, Counter
import sklearn.metrics
from scipy.spatial.distance import pdist, squareform
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models.ldamulticore import LdaMulticore
import nltk
import pyLDAvis
import pyLDAvis.gensim
#nltk.download()

C:\Users\yanxi\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
# read n-grams
data = joblib.load('data/ngrams')
# reset index
data.reset_index(inplace=True)

In [5]:
# join bi-grams and tri-grams with a space
data['reviews_bi'] = data.apply(lambda row: {' '.join(item[0]):item[1] for item in row['reviews_bi'].items()}, axis=1)
data['contents_bi'] = data.apply(lambda row: {' '.join(item[0]):item[1] for item in row['contents_bi'].items()}, axis=1)
data['reviews_tri'] = data.apply(lambda row: {' '.join(item[0]):item[1] for item in row['reviews_tri'].items()}, axis=1)
data['contents_tri'] = data.apply(lambda row: {' '.join(item[0]):item[1] for item in row['contents_tri'].items()}, axis=1)

In [6]:
# combine n-grams
data['ngrams'] = data.apply(lambda row: Counter(row['reviews_mono'])+Counter(row['contents_mono'])+Counter(row['reviews_bi'])+Counter(row['contents_bi'])+Counter(row['reviews_tri'])+Counter(row['contents_tri']), axis=1)


In [7]:
data.head()

,index,title,year,reviews_mono,contents_mono,reviews_bi,contents_bi,reviews_tri,contents_tri,ngrams
0,0,Dead Awake,2016,"{'name': 1, 'last': 1, 'yeari': 1, 'familiar':...","{'dead': 2, 'awak': 2, 'supernatur': 2, 'writt...","{'dream movi': 1, 'right right': 1, 'best thri...","{'world premier': 1, 'psycholog horror': 1, 'r...","{'good horror film': 1, 'good job movi': 1, 'w...","{'refer extern link': 1, 'extern link offici':...","{'name': 1, 'last': 1, 'yeari': 1, 'familiar':..."
1,10,A Good American,2015,"{'situat': 2, 'idea': 1, 'heard': 1, 'read': 2...","{'2015': 1, 'austrian': 1, 'documentari': 3, '...","{'make sens': 1, 'compel film': 1, 'good ameri...","{'produc direct': 1, 'score film': 1, 'refer e...",{},"{'new york time': 1, 'refer extern link': 1, '...","{'situat': 2, 'idea': 2, 'heard': 1, 'read': 2..."
2,11,Hard Tide,2015,"{'watch': 4, 'rot': 1, 'tempt': 1, 'clueless':...","{'hard': 3, 'tide': 3, '2015': 2, 'british': 1...","{'want good': 1, 'watch film': 1, 'year film':...","{'total film': 1, 'hard time': 1, '35 star': 1...","{'doesnt take long': 1, 'nine year old': 1, 'm...","{'gave film posit': 1, 'recept rotten tomato':...","{'watch': 5, 'rot': 1, 'tempt': 1, 'clueless':..."
3,13,Carrie Pilby,2016,"{'fest': 1, 'hyper': 1, 'verbal': 1, 'geniu': ...","{'carri': 5, 'pilbi': 6, 'susan': 3, 'johnson'...","{'good role': 1, 'watch film': 1, 'go see': 1,...","{'film star': 1, 'refer extern': 1, 'film dire...","{'excit see film': 1, 'toronto film festiv': 1...","{'acquir distribut right': 1, 'right film film...","{'fest': 1, 'hyper': 1, 'verbal': 1, 'geniu': ..."
4,14,A Dark Song,2016,"{'writer': 2, 'felt': 2, 'job': 1, 'review': 7...","{'dark': 6, 'song': 6, '2016': 5, 'irish': 3, ...","{'vast major': 1, 'fantast film': 1, 'act plot...","{'end definit': 1, 'festiv releas': 1, 'refer ...","{'good horror film': 1, 'act pretti good': 1, ...","{'film festiv releas': 1, 'film receiv posit':...","{'writer': 2, 'felt': 2, 'job': 1, 'review': 9..."


In [8]:
# create a dictionary indexing the unique terms:
keys = [list(i.keys()) for i in data['ngrams']]
dictionary = corpora.Dictionary(keys)

In [20]:
from numba import jit

C:\Users\yanxi\Anaconda3\lib\site-packages\numba\annotations\type_annotations.py:244: DeprecationWarning: invalid escape sequence \s
  re_longest_white_prefix = re.compile('^\s*')
C:\Users\yanxi\Anaconda3\lib\site-packages\numba\appdirs.py:130: DeprecationWarning: invalid escape sequence \D
  """
C:\Users\yanxi\Anaconda3\lib\site-packages\numba\appdirs.py:235: DeprecationWarning: invalid escape sequence \P
  """


In [21]:
@jit
def get_key(v):
    """ find key based on value"""
    for key, value in dictionary.items():
        if value==v:
            return key
# vectorize function        
get_key_ = np.vectorize(get_key)

In [22]:
data.head()

,index,title,year,reviews_mono,contents_mono,reviews_bi,contents_bi,reviews_tri,contents_tri,ngrams
0,0,Dead Awake,2016,"{'name': 1, 'last': 1, 'yeari': 1, 'familiar':...","{'dead': 2, 'awak': 2, 'supernatur': 2, 'writt...","{'dream movi': 1, 'right right': 1, 'best thri...","{'world premier': 1, 'psycholog horror': 1, 'r...","{'good horror film': 1, 'good job movi': 1, 'w...","{'refer extern link': 1, 'extern link offici':...","{'name': 1, 'last': 1, 'yeari': 1, 'familiar':..."
1,10,A Good American,2015,"{'situat': 2, 'idea': 1, 'heard': 1, 'read': 2...","{'2015': 1, 'austrian': 1, 'documentari': 3, '...","{'make sens': 1, 'compel film': 1, 'good ameri...","{'produc direct': 1, 'score film': 1, 'refer e...",{},"{'new york time': 1, 'refer extern link': 1, '...","{'situat': 2, 'idea': 2, 'heard': 1, 'read': 2..."
2,11,Hard Tide,2015,"{'watch': 4, 'rot': 1, 'tempt': 1, 'clueless':...","{'hard': 3, 'tide': 3, '2015': 2, 'british': 1...","{'want good': 1, 'watch film': 1, 'year film':...","{'total film': 1, 'hard time': 1, '35 star': 1...","{'doesnt take long': 1, 'nine year old': 1, 'm...","{'gave film posit': 1, 'recept rotten tomato':...","{'watch': 5, 'rot': 1, 'tempt': 1, 'clueless':..."
3,13,Carrie Pilby,2016,"{'fest': 1, 'hyper': 1, 'verbal': 1, 'geniu': ...","{'carri': 5, 'pilbi': 6, 'susan': 3, 'johnson'...","{'good role': 1, 'watch film': 1, 'go see': 1,...","{'film star': 1, 'refer extern': 1, 'film dire...","{'excit see film': 1, 'toronto film festiv': 1...","{'acquir distribut right': 1, 'right film film...","{'fest': 1, 'hyper': 1, 'verbal': 1, 'geniu': ..."
4,14,A Dark Song,2016,"{'writer': 2, 'felt': 2, 'job': 1, 'review': 7...","{'dark': 6, 'song': 6, '2016': 5, 'irish': 3, ...","{'vast major': 1, 'fantast film': 1, 'act plot...","{'end definit': 1, 'festiv releas': 1, 'refer ...","{'good horror film': 1, 'act pretti good': 1, ...","{'film festiv releas': 1, 'film receiv posit':...","{'writer': 2, 'felt': 2, 'job': 1, 'review': 9..."


In [ ]:
# map n-grams with keys
data['tokens'] = data.apply(lambda row: list(zip(get_key_(list(row['ngrams'].keys())), row['ngrams'].values())), axis=1)

In [ ]:
data.head()

In [ ]:
corpus = list(data['tokens'])

In [44]:
## Create a dictionary indexing the unique terms:
#dictionary = corpora.Dictionary(texts)

## store the dictionary, for future reference
#dictionary.save('wiki.dict') 

In [57]:
## Create a sparsely formatted corpus:
#corpus = [dictionary.doc2bow(text) for text in texts]
#
## Store to disk, for later use:
#corpora.MmCorpus.serialize('wiki.mm', corpus)  

In [ ]:
# Specify a number of topics:
K = 50

In [ ]:
# Fit the LDA model (100 topics, 10 passes takes about 1/2 hour, 3 topics 3 passes takes 5 min):
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=K, id2word = dictionary, passes=20, alpha=.1/K, eta=.1/K)

# Save the model object for visualization:
ldamodel.save('wiki.model')

In [149]:
# Collect phi matrix of topic word proportions, theta matrix of document topic proportions, f_v of word frequencies
# p_v of term probabilities, V_n of unique terms in each document, W_v of document lengths, f_k of number of words 
# used in each topic, and p_k probabilities of each topic occurring, and bayes probabilities of a topic given a word,
# and a vocab list of terms:
phi_kv = np.zeros((K, len(dictionary)))
theta_nk = np.zeros((len(corpus), K))
for k in range(0,K):
    phi_kv[k,:] = [word_prob[1] for word_prob in lda_mult.get_topic_terms(k, len(dictionary))]

In [150]:
phi_kv.shape

(50, 33346)

In [151]:
theta_nk.shape

(3098, 50)

In [152]:
ls = np.zeros([len(corpus),1])
for n in range(0,len(corpus)):
    topic_dists = ldamodel.get_document_topics(corpus[n], minimum_probability=0)
    for topic_dist in topic_dists:
        theta_nk[n, topic_dist[0]] = topic_dist[1]

In [153]:
counts = Counter()
for text in texts:
    for word in text:
        counts[word] += 1
f_v = np.array([counts[dictionary[word_ind]] for word_ind in range(len(dictionary))])
p_v = f_v/sum(f_v)
V_n = np.array([len(doc) for doc in corpus])
W_n = np.array([sum(word[1] for word in doc) for doc in corpus]) 
f_k = [sum(theta_nk[:,k]*W_n) for k in range(K)]
p_k = f_k/sum(f_k)
bayes_kv = np.zeros((K, len(dictionary)))
for k in range(K):
    bayes_kv[k,:] = phi_kv[k,:]*p_k[k]/p_v
vocab = [dictionary[i] for i in range(len(dictionary))]

In [154]:
data = {'topic_term_dists': phi_kv, 
            'doc_topic_dists': theta_nk,
            'doc_lengths': W_n,
            'vocab': vocab,
            'term_frequency': f_v}

In [155]:
movies_vis_data = pyLDAvis.prepare(**data)


C:\Users\yanxi\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [156]:
pyLDAvis.display(movies_vis_data)


In [139]:
# Specify number of top words:
num_top_words = 10

# Show top words from each topic:
for k in range(K):
    print("topic " + str(k) + ":")
    topic_top_words = ldamodel.show_topic(k)
    for top_word in topic_top_words:
        print((top_word[0],format(top_word[1],".2f")))
    print("\n")

topic 0:
('queen', '0.02')
('film', '0.02')
('diana', '0.01')
('royal', '0.01')
('blair', '0.01')
('famili', '0.01')
('mirren', '0.01')
('bronson', '0.01')
('elizabeth', '0.01')
('helen', '0.01')


topic 1:
('film', '0.02')
('et', '0.02')
('movi', '0.02')
('spielberg', '0.01')
('maria', '0.01')
('stori', '0.01')
('lincoln', '0.01')
('like', '0.01')
('time', '0.01')
('bfg', '0.01')


topic 2:
('ami', '0.03')
('comedi', '0.02')
('funni', '0.01')
('chef', '0.01')
('special', '0.01')
('steve', '0.01')
('day', '0.01')
('show', '0.01')
('murphi', '0.01')
('joke', '0.01')


topic 3:
('movi', '0.00')
('film', '0.00')
('like', '0.00')
('time', '0.00')
('good', '0.00')
('watch', '0.00')
('charact', '0.00')
('realli', '0.00')
('make', '0.00')
('stori', '0.00')


topic 4:
('movi', '0.04')
('film', '0.02')
('good', '0.01')
('end', '0.01')
('see', '0.01')
('like', '0.01')
('realli', '0.01')
('watch', '0.01')
('act', '0.01')
('time', '0.01')


topic 5:
('mulan', '0.05')
('seagal', '0.04')
('chines', 

('prison', '0.03')
('film', '0.02')
('murray', '0.01')
('guard', '0.01')
('kyle', '0.01')
('get', '0.01')
('gibson', '0.01')
('rock', '0.01')
('play', '0.01')
('mel', '0.01')


topic 45:
('babi', '0.13')
('michel', '0.07')
('mari', '0.03')
('obama', '0.02')
('presid', '0.02')
('child', '0.02')
('first', '0.02')
('pregnant', '0.01')
('birth', '0.01')
('polit', '0.01')


topic 46:
('shark', '0.05')
('jaw', '0.02')
('film', '0.02')
('movi', '0.02')
('brodi', '0.01')
('dori', '0.01')
('first', '0.01')
('water', '0.01')
('great', '0.01')
('spielberg', '0.01')


topic 47:
('film', '0.03')
('tarantino', '0.03')
('movi', '0.02')
('pulp', '0.01')
('fiction', '0.01')
('scene', '0.01')
('charact', '0.01')
('quentin', '0.01')
('best', '0.01')
('nazi', '0.01')


topic 48:
('kirbi', '0.74')
('gone', '0.03')
('far', '0.03')
('armani', '0.02')
('oc', '0.01')
('film', '0.00')
('movi', '0.00')
('like', '0.00')
('charact', '0.00')
('time', '0.00')


topic 49:
('movi', '0.04')
('film', '0.02')
('like', '0

('movi', '0.03')
('king', '0.02')
('book', '0.02')
('alien', '0.02')
('stephen', '0.01')
('film', '0.01')
('read', '0.01')
('dreamcatch', '0.01')
('freeman', '0.01')
('like', '0.01')


topic 88:
('movi', '0.02')
('film', '0.02')
('stiller', '0.01')
('chicago', '0.01')
('music', '0.01')
('actor', '0.01')
('like', '0.01')
('downey', '0.01')
('charact', '0.01')
('perform', '0.01')


topic 89:
('lee', '0.05')
('seri', '0.05')
('island', '0.05')
('phil', '0.04')
('brian', '0.04')
('tommi', '0.03')
('oklahoma', '0.02')
('rugrat', '0.02')
('spike', '0.02')
('flint', '0.02')


topic 90:
('jack', '0.04')
('film', '0.02')
('kubrick', '0.02')
('movi', '0.02')
('shine', '0.02')
('hotel', '0.01')
('nicholson', '0.01')
('horror', '0.01')
('danni', '0.01')
('king', '0.01')


topic 91:
('pete', '0.03')
('dragon', '0.02')
('hunter', '0.01')
('holli', '0.01')
('wuorno', '0.01')
('film', '0.01')
('pilot', '0.01')
('elliot', '0.01')
('aileen', '0.01')
('nate', '0.01')


topic 92:
('documentari', '0.04')
(

In [140]:
# Obtain topic distribution for each movie review and every movie content:
topic_probs = []
for document in corpus:
    topic_probs.append(np.array([topic_probs_double[1] for topic_probs_double in ldamodel.get_document_topics(document, minimum_probability=0)]))
topic_probs_array = np.asarray(topic_probs)
topic_probs_array

array([ array([  1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   8.39712646e-02,   1.05585471e-06,
         1.05585471e-06,   1.26769899e-02,   2.57129858e-02,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   4.55243515e-02,
         1.05585471e-06,   4.87939017e-03,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585471e-06,   1.05585471e-06,   1.05585471e-06,
         1.05585

In [145]:
# Compute pairwise negative hellinger distance matrix for documents:
#document_similarities = calc_pairwise_sims_matrix(topic_probs_array, "hellinger")
# Compute pairwise cosine similarity matrix for documents:
document_similarities = calc_pairwise_sims_matrix(topic_probs_array, "cosine")

AttributeError: module 'sklearn.metrics' has no attribute 'calc_pairwise_sims_matrix'

In [142]:
# Return the 10 documents most similar to a specified movie:
movie_title = "The Search for Santa Paws"
titanic_index = movie_data.loc[movie_data['title'] == movie_title].index[0:2][0]
titanic_similarities = document_similarities[titanic_index, :]
closest_movies = sorted(range(len(titanic_similarities)), key=lambda k: -titanic_similarities[k])
movie_data['title'].loc[closest_movies]

NameError: name 'document_similarities' is not defined

In [ ]:
# Compute matrix of topic distribution similarities:
#topic_similarity_matrix = np.zeros((len(corpus), len(corpus)))
#for i in range(5):
#    for j in range(5):
#        topic_similarity_matrix[i,j] = gensim.matutils.cossim([topic_probs_double[1] for topic_probs_double in ldamodel.get_document_topics(corpus[i], minimum_probability=0)], [topic_probs_double[1] for topic_probs_double in ldamodel.get_document_topics(corpus[j], minimum_probability=0)])


In [ ]:
# Obtain topic distribution for each movie review and every movie content:
#topic_probs = []
#for document in corpus:
#    topic_probs.append(np.array([topic_probs_double[1] for topic_probs_double in lda_mult.get_document_topics(document, minimum_probability=0)]))
#topic_probs_array = np.asarray(topic_probs)
#topic_probs_array

In [ ]:
# TO DO: FIND TOP WORDS BY BAYES RULE:

In [ ]:
# Compute tf-idf for each document:
tf_idf = gensim.models.TfidfModel(corpus)

# Compute similarity for each document pair:
document_similarities = gensim.similarities.Similarity('doc_sims',tf_idf[corpus],
                                      num_features=len(dictionary))

# Find the similarity of movies to a query movie:
query_doc = [w.lower() for w in texts[157]]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)
sims[query_doc_tf_idf]

In [14]:
# Fit the LDA model with multi core (3 topics 3 iterations 3 passes, 4 min):
lda_mult = LdaMulticore(corpus, num_topics=K, id2word=dictionary, iterations = 3, passes = 3, alpha=.1/K, eta=.1/K)

In [ ]:
# Store the corpus in Blei's LDA-C format:
#corpora.BleiCorpus.serialize('wiki.lda-c', corpus)

# Read in the corpus in Blei's LDA-C format:
#corpus_ldac = corpora.BleiCorpus('wiki.lda-c')

In [ ]:
# format time slices:
#time_slices = {i: 0 for i in range(1939,2018)}
#for year in movie_data['year']:
#    time_slices[year] += 1
#time_slices = [val for val in time_slices.values()]
#time_slices

In [ ]:
# Run dynamic lda on the wikipedia and imdb corpus:
#model = DtmModel('dtm-win64.exe', corpus_ldac, time_slices, num_topics=3, id2word=dictionary)